# Create an RK05 Disk of Tempest Sources

### First Some Radix-50 Helper Functions
This is an encoding used by the PDP-11 for file names.

In [1]:
import math

CHARS = " ABCDEFGHIJKLMNOPQRSTUVWXYZ$.*0123456789"

def radix50_decode(v):
    if v > 63999:
        raise Exception("Invalid Value")
    c1 = math.floor(v/(40**2))
    v2 = v%(40**2)
    c2 = math.floor(v2/40)
    c3 = v2%40
    return CHARS[c1]+CHARS[c2]+CHARS[c3]

def radix50_encode(chars):
    if len(chars) != 3:
        raise Exception("Must be 3 characters")
    result = ((CHARS.index(chars[0]) * (40**2)) + 
              (CHARS.index(chars[1]) * (40)) + 
              CHARS.index(chars[2]))
    return result

print(radix50_encode("ABC"),radix50_encode("ABC") == 1683)
print(radix50_encode("DEF"),radix50_encode("ABC") == 6606)

print(radix50_decode(1683),radix50_decode(1683) == "ABC")
print(radix50_decode(6606),radix50_decode(6606) == "DEF")


1683 True
6606 False
ABC True
DEF True


### Create an rk05 image file with the tempest source files

In [154]:
!git clone https://github.com/historicalsource/tempest

Cloning into 'tempest'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 45 (delta 8), reused 45 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 174.01 KiB | 2.49 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [2]:
import os

BLOCK_LENGTH = 512
FILES_START = 0x4c00
word = lambda x: (x).to_bytes(2,"little")

# Read in each source file from the git repo we cloned
SRC_FOLDER = "tempest"
files_to_write = []
for file_name in os.listdir(SRC_FOLDER):
    if ".git" in file_name:
        continue
    data = open(SRC_FOLDER+'/'+file_name,'rb').read().strip().strip(b'\x00')
    files_to_write += [(file_name,data,len(data))]

# Copy in the home block header from another disk image
rk1 = open("rk1.rk05",'rb')
header = rk1.read(0xc00)

# Header for the directory entry
dir_entry = word(16) + word(0) + word(1) + word(0) + word(38)
# The file data for the main part of the disk
file_data = b''

# Write a directory entry for each file and append its zero-padded data
# to file_data for writing later
for (filename,data,file_len) in files_to_write:
    #filetype
    dir_entry += word(0x0400)
    
    #filename
    filename = filename.split('.')
    trio1 = filename[0][:3].ljust(3)
    trio2 = filename[0][3:].ljust(3)
    trio3 = filename[1].ljust(3)
    dir_entry += word(radix50_encode(trio1))
    dir_entry += word(radix50_encode(trio2))
    dir_entry += word(radix50_encode(trio3))

    # No of blocks used by file
    blocks = math.floor(file_len/BLOCK_LENGTH) + 1
    dir_entry += word(blocks)
    
    #reserved
    dir_entry += word(0)
    #filedate
    dir_entry += word(0)

    # add the file data
    padding = b'\x00' * ((blocks*BLOCK_LENGTH) - file_len)
    file_data += data + padding

# Add the end segment to the end of the directory structure
dir_entry += word(0x0800)

# Padding for the directory block
total_so_far = len(header + dir_entry)
dir_entry_padding = b'\x00' * (FILES_START - total_so_far)

# Padding for the main block
main_data = header + dir_entry + dir_entry_padding + file_data
#main_padding = b'\x00' * (0x261200 - len(main_data))

# Write everything to our image file
open("tempest_original.rk05",'wb').write(main_data)

#!xxd tempest_original.rk05

585728